<a href="https://colab.research.google.com/github/monicahatis/Atari-with-deep-learning/blob/main/SpaceInvaders.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Installing and acquiring the required libraries
!pip install tensorflow==2.3.1 gym keras-rl2 gym[atari]
!pip install gym[accept-rom-license]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 320.4 MB 26 kB/s 
     |████████████████████████████████| 52 kB 1.1 MB/s 
     |████████████████████████████████| 20.1 MB 1.2 MB/s 
     |████████████████████████████████| 459 kB 54.5 MB/s 
     |████████████████████████████████| 2.9 MB 47.1 MB/s 
  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.6
    Uninstalling numpy-1.21.6:
      Successfully uninstalled numpy-1.21.6
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.8.0
    Uninstalling tensorflow-estimator-2.8.0:
      Successfully uninstalled tensorflow-estimator-2.8.0
  Attempting uninstall: h5py
    Found existing installation: h5py 3.1.0
    Uninstalling h5py-3.1.0:
      Successfully uninstalled h5py-3.1.0
  Attempting uninstall: gast
    Found existing installation: gast 0.5.3
    Uninstalling gast-0.5.3:
      Su

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#installing the OpenAI Gym environment
%pip install -U gym>=0.21.0
%pip install -U gym[atari,accept-rom-license]


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.6 MB 15.0 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for AutoROM.accept-rom-license: filename=AutoROM.accept_rom_license-0.4.2-py3-none-any.whl size=441027 sha256=7ba52422c60db66f2e4f273270601fcc8fc38c9e747df9f077ee7f11ac1d86f1
  Stored in directory: /root/.cache/pip/wheels/87/67/2e/6147e7912fe37f5408b80d07527dab807c1d25f5c403a9538a
Successfully built AutoROM.accept-rom-license


In [ ]:
#installing a wrapper for rendering OpenAI Gym environments in Google Colab
!pip install colabgymrender

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for colabgymrender: filename=colabgymrender-1.1.0-py3-none-any.whl size=3132 sha256=86164541abb00af91ca64236bd1a42d1c49b6633133e10e3992114c1f1ef7b3f
  Stored in directory: /root/.cache/pip/wheels/f1/0a/2a/86955ea711b461ab7918236fed2568733f75ed677d0524b56c
Successfully built colabgymrender


In [ ]:

import gym #setting up the game environment
import random #take random steps in the game
from colabgymrender.recorder import Recorder #to record a video of the game playing in the background

Imageio: 'ffmpeg-linux64-v3.3.1' was not found on your computer; downloading it now.
Try 1. Download from https://github.com/imageio/imageio-binaries/raw/master/ffmpeg/ffmpeg-linux64-v3.3.1 (43.8 MB)
Downloading: 8192/45929032 bytes (0.0%)3530752/45929032 bytes (7.7%)7208960/45929032 bytes (15.7%)10862592/45929032 bytes (23.7%)14647296/45929032 bytes (31.9%)18358272/45929032 bytes (40.0%)22061056/45929032 bytes (48.0%)25616384/45929032 bytes (55.8%)29376512/45929032 bytes (64.0%)33169408/45929032 bytes (72.2%)36478976/45929032 bytes (79.4%)40181760/45929032 bytes (87.5%)43859968/45929032 bytes (95.5%)

In [ ]:
#setting up the gym environment for space invaders 
#setting up the video recording environment


env = gym.make('SpaceInvaders-v0')
env = Recorder(env, './video')
height, width, channels = env.observation_space.shape
actions = env.action_space.n 


In [ ]:
env.unwrapped.get_action_meanings() # to see the possible actions in the game

['NOOP', 'FIRE', 'RIGHT', 'LEFT', 'RIGHTFIRE', 'LEFTFIRE']

In [ ]:
#run the game before training for five episodes
episodes = 5
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
      # taking random actions and extracting the result which is score plus reward
        env.render(mode='rgb_array')
        action = random.choice([0,1,2,3,4,5])
        n_state, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))
env.close()

Episode:1 Score:420.0
Episode:2 Score:120.0
Episode:3 Score:105.0
Episode:4 Score:185.0
Episode:5 Score:105.0


In [ ]:
#importing tools for the neural network model
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Convolution2D
from tensorflow.keras.optimizers import Adam

In [ ]:
#adding layers to the model
def build_model(height, width, channels, actions):
    model = Sequential()
    model.add(Convolution2D(32, (8,8), strides=(4,4), activation='relu', input_shape=(3,height, width, channels)))
    model.add(Convolution2D(64, (4,4), strides=(2,2), activation='relu'))
    model.add(Convolution2D(64, (3,3), activation='relu'))
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model

In [ ]:
model = build_model(height, width, channels, actions)

In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 3, 51, 39, 32)     6176      
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 3, 24, 18, 64)     32832     
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 3, 22, 16, 64)     36928     
_________________________________________________________________
flatten_1 (Flatten)          (None, 67584)             0         
_________________________________________________________________
dense_4 (Dense)              (None, 512)               34603520  
_________________________________________________________________
dense_5 (Dense)              (None, 256)               131328    
_________________________________________________________________
dense_6 (Dense)              (None, 6)                

In [ ]:
#importing tools for the agent
from rl.agents import DQNAgent
from rl.memory import SequentialMemory
from rl.policy import LinearAnnealedPolicy, EpsGreedyQPolicy

In [ ]:
#construct a function for building an agent similar to our model construction.
def build_agent(model, actions):
    policy = LinearAnnealedPolicy(EpsGreedyQPolicy(), attr='eps', value_max=1., value_min=.1, value_test=.2, nb_steps=7500)
    memory = SequentialMemory(limit=1000, window_length=3)
    DeepQN = DQNAgent(model=model, memory=memory, policy=policy,enable_dueling_network=True, dueling_type='avg', nb_actions=actions, nb_steps_warmup=1000)
    return DeepQN

In [ ]:
#build the agent
DeepQN = build_agent(model, actions)
DeepQN.compile(Adam(lr=1e-4))

In [ ]:
#training the model
DeepQN.fit(env, nb_steps=7500, visualize=False, verbose=2)

Training for 7500 steps ...
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
  523/7500: episode: 1, duration: 22.888s, episode steps: 523, steps per second:  23, episode reward: 125.000, mean reward:  0.239 [ 0.000, 25.000], mean action: 2.499 [0.000, 5.000],  loss: --, mean_q: --, mean_eps: --
 1055/7500: episode: 2, duration: 139.188s, episode steps: 532, steps per second:   4, episode reward: 80.000, mean reward:  0.150 [ 0.000, 25.000], mean action: 2.412 [0.000, 5.000],  loss: 26.272626, mean_q: 9.143638, mean_eps: 0.876700
 2132/7500: episode: 3, duration: 2453.001s, episode steps: 1077, steps per second:   0, episode reward: 390.000, mean reward:  0.362 [ 0.000, 30.000], mean action: 2.544 [0.000, 5.000],  loss: 1.575442, mean_q: 6.171043, mean_eps: 0.808840
 2783/7500: episode: 4, duration: 1476.971s, episode steps: 651, steps per second:   0, episode reward: 75.000, mean reward:  0.115 [ 0.000, 25.000], mean 

In [ ]:
#testing our model after training by running few episodes
scores = DeepQN.test(env, nb_episodes=10, visualize=False)
print(np.mean(scores.history['episode_reward']))
env.play()

Testing for 10 episodes ...
Episode 1: reward: 185.000, steps: 569
Episode 2: reward: 185.000, steps: 576
Episode 3: reward: 185.000, steps: 579
Episode 4: reward: 385.000, steps: 1110
Episode 5: reward: 185.000, steps: 585
Episode 6: reward: 185.000, steps: 575
Episode 7: reward: 195.000, steps: 577
Episode 8: reward: 255.000, steps: 706
Episode 9: reward: 185.000, steps: 573
Episode 10: reward: 305.000, steps: 772
225.0


In [ ]:
DeepQN.save_weights('SavedWeights/7k-Fast/DeepQN_weights.h5f')